In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt
%matplotlib inline
from mpl_toolkits.axes_grid1 import ImageGrid


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

i = 0
fig = plt.figure(1, (30, 30))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(7, 3),  # creates 2x2 grid of axes
                 axes_pad=0.2,  # pad between axes in inch.
                 )


print("Total images :" , len(images))
# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        ax = grid[i]
        ax.imshow(img)
        i = i +1
plt.show()


#cv2.destroyAllWindows()

#print(ret)
#print(corners)
print("Done")

If the above cell ran sucessfully, you should now have objpoints and imgpoints needed for camera calibration. Run the cell below to calibrate, calculate distortion coefficients, and test undistortion on an image!

In [ ]:
import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('camera_cal/test_image.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)


dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('camera_cal/test_undist.jpg',dst)

# Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

## Perspective transform

In [ ]:
#import pickle
#import cv2
#import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg

# Read in the saved camera matrix and distortion coefficients
# These are the arrays you calculated using cv2.calibrateCamera()
dist_pickle = pickle.load( open( "camera_cal/wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Read in an image
img = cv2.imread('test_images/straight_lines1.jpg')
nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y

# MODIFY THIS FUNCTION TO GENERATE OUTPUT 
# THAT LOOKS LIKE THE IMAGE ABOVE

print(dist)
plt.imshow(img)

img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

plt.figure()
plt.imshow(img)

def corners_unwarp(img, nx, ny, mtx, dist):
    # 1) Undistort using mtx and dist.
    undis = cv2.undistort(img, mtx, dist, None, mtx)
    # 2) Convert to grayscale
    plt.figure()
    plt.imshow(undis)
    
    gray = cv2.cvtColor(undis, cv2.COLOR_BGR2GRAY)
    # 3) Find the chessboard corners
    offset=10
    
    src_bottom_left = [260,680]
    src_bottom_right = [1040,680]
    src_top_left = [581,460]
    src_top_right = [700,460]
    
    destination_bottom_left = [100,700]
    destination_bottom_right = [1000,700]
    destination_top_left = [100,50]
    destination_top_right = [1000,50]
    
    #src = np.float32([corners[0],corners[nx-1],corners[-1],corners[-nx]])
    src = np.float32([[src_top_left,src_top_right,src_bottom_right,src_bottom_left]])
    print(src)
    print(img.shape)
    dst_points = np.float32([[destination_top_left,destination_top_right,destination_bottom_right,destination_bottom_left]])
    #dst_points = np.float32([[offset,offset],
    #                [img.shape[1]-offset,offset],
    #                [img.shape[1]-offset,img.shape[0]-offset],
    #                [offset,img.shape[0]-offset]])
    #dst_points = np.float32([(offset,offset),(1200-offset,offset),()])
    M = cv2.getPerspectiveTransform(src, dst_points)
    warped = cv2.warpPerspective(undis, M, (img.shape[1],img.shape[0]), flags=cv2.INTER_LINEAR)

    #delete the next two lines
    #M = None
    #warped = np.copy(img) 
    return warped, M

top_down, perspective_M = corners_unwarp(img, nx, ny, mtx, dist)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down)
ax2.set_title('Undistorted and Warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
